# 第九章 同步

## 9.1 背景

**独立的线程**：不和其他线程共享资源或状态，它具有确定性（输入状态决定结果）和可重现性（能够重现起始条件的I/O）。

**合作的线程**：在多个线程中共享状态，它具有不确定性和不可重现性。

**同步的目标**：无论多个线程的指令序列怎样交替执行，程序都必须正常工作。

**进程/线程同步的优点**：资源共享、加速、模块化。

## 9.2 概念

**竞态条件（Race Condition）**：结果依赖于并发执行或者事件的顺序/时间。

**原子操作（Atomic Operation）**：一次不存在任何中断或者失败的执行。

**临界区（Critical Section）**：进程中的一段需要访问共享资源并且当另一个进程处于相应代码区域时便不会执行的代码区域。

**互斥（Mutual Exclusion）**：当一个进程处于临界区并访问共享资源时，没有其他进程会处于临界区并且访问任何相同的资源。

**死锁（Dead Lock）**：两个或以上的进程，在相互等待完成特定任务，而最终没法将自身任务进行下去。

**饥饿（Starvation）**：一个可执行的进程被调度器持续忽略，以至于虽然处于可执行状态却不被执行。

## 9.3 临界区

**临界区属性**：
* 互斥：同一时间内，临界区中最多存在一个线程。
* 前进（Progress）：如果一个线程想要进入临界区，那么它最终会成功。
* 有限等待：如果一个线程i处于入口区，那么在i的请求被接收之前，其他线程进入临界区的时间是有限的。
* 无忙等待（可选）：如果一个进程在等待进入临界区，可以在进入前被挂起。

## 9.4 方法1：使用硬件中断

**使用硬件中断的方式实现同步**：没有中断，没有上下文切换，因此没有并行，硬件将中断处理延迟到中断被启用之后。

**缺点**：
* 一旦中断被禁用，线程就无法被停止。
* 要死临界区可以任意长，处理器将无法正常工作。
* 无法营养在多CPU的机器。

## 9.5 方法2：基于软件的解决方案

### 9.5.1 两个进程间的互斥

**尝试1**：满足互斥，但是有时不满足前进性。$T_0$执行后，turn=1，但是$T_1$并不执行，此时$T_0$无法再次获取资源。

``` C++
int turn = 0;

// Thread i
turn = i;
do {
    while(turn != i);
        <critical section>
    turn = j;
        <reminder section>
} while(1);
```

**尝试2**：虽然满足前进性，但是不满足互斥。

``` C++
int flag[2] = {0};

// Thread i
flag[i] = 1;
do {
    while(flag[j] == 1);
    flag[i] = 1;
        <critical section>
    flag[i] = 0;
        <reminder section>
} while(1);
```

**尝试3**：满足互斥，但是存在死锁。

``` C++
int flag[2] = {0};

// Thread i
do {
    flag[i] = 1;
    while(flag[j] == 1);
        <critical section>
    flag[i] = 0;
        <reminder section>
} while(1);
```

**Peterson 算法**：满足互斥、前进、有限等待等性质。

``` C++
int turn; // 指示该哪个进程进入临界区
boolean flag[2]; // 指示进程是否准备好进入临界区

// Code for Enter Critical Section
flag[i] = TRUE;
turn = j;
while (flag[j] && turn[j]);

// Code for Exit Critical Section
flag[i] = FALSE;
```

**Dekker 算法**和**Eisenberg and Mcguire's 算法**都是解决两个进程竞态条件的算法。

## N进程的临界区

**Bakery 算法**：
* 进入临界区之前，进程接收到一个数字。
* 得到数字最小的进入临界区。
* 如果进程$P_i$和$P_j$收到相同的数字，那么如果 $i<j$，则$P_i$先进。

## 9.6 方法3：更高级的抽象

**实现机制**：
* 硬件提供了一些原语（原子操作的指令）。
* 操作系统提供了更高级的编程抽象来简化并行程序。

**锁**：一个抽象的数据结构来解决同步问题。它的**底层支持**是两个原语指令：
* Test-and-Set 测试和重置
    * 从内存中读取值
    * 测试该值。
    * 内存值设置为1。
* 交换：交换内存中的两个值。

``` C++
boolean TestAndSet(boolean* target) {
    boolean rv = *target;
    *target = 1;
    return rv;
}

void Exchange(boolean* a, boolean* b) {
    boolean temp = *a;
    *a = *b;
    *b = temp;
}
```

**通过Test-and-Set实现锁**

``` C++
class Lock {
    int value = 0;
    
    void Acquire() {
        while(test-and-set(value) == 1);
    }
    
    void Release() {
        value = 0;
    }
};
```

上述方法的缺点是使用了忙等待的方式，如果加入调度队列，可以优化该点：


``` C++
class Lock {
    int value = 0;
    waitQueue q;
    
    void Acquire() {
        while(test-and-set(value) == 1);
        <add this TCB to wait queue q>
        schedule();
    }
    
    void Release() {
        value = 0;
        <remove thread from q>
        wakeup();
    }
};
```

**通过交换实现锁**


``` C++
int lock = 0;

// Thread i
int key = 1;
do {
    key = 1;
    while (key == 1) exchange(lock, key);
        <critical section>
    lock = 0;
        <reminder section>
while(1);
```


**高级抽象的优点**：
* 适用于单处理器或共享主存的多处理器中任意数量的进程。
* 简单并且容易证明。
* 可用于支持多临界区。

**高级抽象的缺点**：
* 忙等待消耗处理器的时间。
* 当进程离开临界区并且多个进程在等待的时候可能导致饥饿。
* 死锁。